# Import the dependent libraries

In [302]:
import panel as pn
pn.extension('plotly')
from panel.interact import interact
from panel import widgets
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
from pathlib import Path
from dotenv import load_dotenv #Just in case we need an API key.
import requests
import json
import numpy as np
%matplotlib inline
from datetime import date
from datetime import timedelta
today = date.today()
yesterday = today - timedelta(days = 1)
start_date = yesterday - timedelta(days = 300)#wont let me get more then 298 days, or so.
start_date = start_date.strftime("%Y-%m-%d")
yesterday = yesterday.strftime("%Y-%m-%d")

## Fetch the data from Coinbase API
- TODO
    - Need to show data before dataframe conversion (show our process before creating function)
    - Explain how we got column names
    - 

In [303]:


def fetch_daily_data(symbol):
    pair_split = symbol.split('/') # Splitting our symbol by the '/' and creating a a list for the new values.
    symbol = pair_split[0] + '-' + pair_split[1] # symbol = BTC-USD #The API request format requires the dash.
    url = f'https://api.pro.coinbase.com/products/{symbol}/candles?start={start_date}&end={yesterday}&granularity=86400'#notice the symbol insert. There are 86400 seconds in a day.
    response = requests.get(url) #getting response from website
    if response.status_code == 200: # check to make sure the response from server is good
        #if response is good then we create a dataframe by reformatting a json load.
        data = pd.DataFrame(json.loads(response.text), columns=['unix', 'low', 'high', 'open', 'close', 'volume'])
        data['date'] = pd.to_datetime(data['unix'], unit='s') # convert to a readable date
       #######

        # if we failed to get any data, print an error...otherwise write the file
        if data is None:
            print("Did not return any data from Coinbase for this symbol")
        else:
            data.to_csv(f'Coinbase_{pair_split[0] + pair_split[1]}_dailydata.csv', index=False)
    else:
        print("Did not receieve OK response from Coinbase API")

## Call the fetch function with a function focused on our three main cryptos: BTC/USD, ETH/USD, LTC/USD.

In [304]:
def fetch_main_cryptos():
  # we set which pair we want to retrieve data for
    pair = "BTC/USD"
    pair = "ETH/USD"
    pair = "LTC/USD"
    fetch_daily_data(symbol=pair)
        #call the function calling our API
fetch_main_cryptos()

## Created a path to our newly created CSV files

In [305]:
BTC_path, ETH_path, LTC_path = (Path('Coinbase_BTCUSD_dailydata.csv'),
                                Path('Coinbase_ETHUSD_dailydata.csv'),
                                Path('Coinbase_LTCUSD_dailydata.csv'))
BTC_df, ETH_df, LTC_df = (pd.read_csv(BTC_path, index_col='date', infer_datetime_format=False, parse_dates=True),
                          pd.read_csv(ETH_path, index_col='date', infer_datetime_format=False, parse_dates=True),
                          pd.read_csv(LTC_path, index_col='date', infer_datetime_format=False, parse_dates=True))

## 1

In [306]:
#This function is to create our main datasets. Please edit and comment on how we should approach this.
def clean_data(df):
    df = df.dropna() # immediately drop any null values
    df = df.drop(columns=['unix']).copy() #create deep copy of df with desired columns
    df['volume_change'] = df['volume'].pct_change() #find daily percent change in volume
    df['percent_volatility'] = round(((df['high'] - df['low']) / df['high']) * 100, 2) #Finding the amount of change between the low and high, then comparing it to the high.
    df['daily_change'] = round(df['close'].pct_change(), 5) # daily pct change
    df.drop(df.head(2).index, inplace=True) # drop the unfinished and upcoming day, inclusive of NA data
    df.sort_index(inplace=True)
    return pd.DataFrame(df)
def clean_new_data(df):
    return df.dropna()
#these are the base data sets so far
BTC_df = clean_data(BTC_df)
ETH_df = clean_data(ETH_df)
LTC_df = clean_data(LTC_df)

## Current Dataframes

In [323]:
BTC_df.head(10000)

,low,high,open,close,volume,volume_change,percent_volatility,daily_change
date,,,,,,,,
2020-03-27,6260.00,6880.00,6760.00,6372.36,20458.244021,0.005128,9.01,0.01928
2020-03-28,6030.00,6372.36,6372.36,6251.82,20353.874846,0.263334,5.37,0.06374
2020-03-29,5870.46,6279.96,6251.45,5877.21,16111.236378,-0.319046,6.52,-0.08260
2020-03-30,5853.00,6631.23,5878.98,6406.40,23659.802642,0.981720,11.74,-0.00279
2020-03-31,6333.91,6524.79,6406.40,6424.35,11939.025331,-0.333025,2.93,-0.03627
...,...,...,...,...,...,...,...,...
2021-01-14,36751.11,40127.66,37393.67,39125.14,31868.760494,-0.124991,8.41,0.06449
2021-01-15,34298.93,39697.00,39123.05,36754.67,36421.059188,0.745857,13.60,0.02077
2021-01-16,35372.59,37948.00,36754.60,36006.94,20861.425452,0.087549,6.79,0.00522


In [308]:
ETH_df.head(2)

,low,high,open,close,volume,volume_change,percent_volatility,daily_change
date,,,,,,,,
2020-03-27,130.00,142.5,139.00,131.56,173786.920120,0.005642,8.77,0.00160
2020-03-28,125.01,133.8,131.56,131.35,172811.936399,0.856847,6.57,0.05629


In [309]:
LTC_df.head(2)

,low,high,open,close,volume,volume_change,percent_volatility,daily_change
date,,,,,,,,
2020-03-27,37.94,41.43,40.60,38.15,249311.220149,-0.066429,8.42,-0.01953
2020-03-28,36.57,39.51,38.17,38.91,267051.300875,1.072342,7.44,0.04850


## Creating new data frames to compare statistical columns

In [369]:
#joined columns into new dataframe and renamed
BTC_volume = BTC_df['volume_change']
ETH_volume = ETH_df['volume_change']
LTC_volume = LTC_df['volume_change']
volume_change_df = clean_new_data(pd.concat([BTC_volume, ETH_volume, LTC_volume], axis=1))
volume_change_df.columns = ['BTC_volume_change', 'ETH_volume_change', 'LTC_volume_change']

#joined columns into new dataframe and renamed
BTC_volatility = BTC_df['percent_volatility']
ETH_volatility = ETH_df['percent_volatility']
LTC_volatility = LTC_df['percent_volatility']
volatility_df = clean_new_data(pd.concat([BTC_volatility, ETH_volatility, LTC_volatility], axis=1))
volatility_df.columns = ['BTC_volatility', 'ETH_volatility', 'LTC_volatility']


#joined columns into new dataframe and renamed
BTC_close = BTC_df['close']
ETH_close = ETH_df['close']
LTC_close = LTC_df['close']
close_df = clean_new_data(pd.concat([BTC_close, ETH_close, LTC_close], axis=1))
close_df.columns = ['BTC_close', 'ETH_close', 'LTC_close']
ETH_LTC_close_df = clean_new_data(close_df.drop(columns='BTC_close'))

#joined columns into new dataframe and renamed
BTC_daily_change = BTC_df['daily_change']
ETH_daily_change = ETH_df['daily_change']
LTC_daily_change = LTC_df['daily_change']
daily_change_df = clean_new_data(pd.concat([BTC_daily_change, ETH_daily_change, LTC_daily_change], axis=1))
daily_change_df.columns = ['BTC_daily_change', 'ETH_daily_change', 'LTC_daily_change']


## Show new Dataframes

In [370]:
volume_change_df.tail(2)

,BTC_volume_change,ETH_volume_change,LTC_volume_change
date,,,
2021-01-17,0.154874,0.025556,-0.253196
2021-01-18,-0.121405,-0.594549,-0.162030


In [371]:
volatility_df.head(1)

,BTC_volatility,ETH_volatility,LTC_volatility
date,,,
2020-03-27,9.01,8.77,8.42


In [372]:
close_df.head(1)

,BTC_close,ETH_close,LTC_close
date,,,
2020-03-27,6372.36,131.56,38.15


In [373]:
ETH_LTC_close_df.head(1)

,ETH_close,LTC_close
date,,
2020-03-27,131.56,38.15


## Create functions for hvplots so they can be combined into a dashboard

In [374]:
#shows us the pct change of daily volume
def volume_change():
    return volume_change_df.hvplot(ylim=(-1, 3))
#shows us trend line of high and low volatility
def volatility():
    return volatility_df.hvplot()
#daily stock prices
def close():
    return close_df.hvplot()
#daily stock prices (ETH and LTC)
def zoom_in_close():
    return ETH_LTC_close_df.hvplot()
#daily pct change
def pct_change_plot():
    return daily_change_df.hvplot()

In [375]:
volume_change_df.hvplot(ylim=(-1, 3))

:NdOverlay   [Variable]
   :Curve   [date]   (value)

In [376]:
volatility_df.hvplot()

:NdOverlay   [Variable]
   :Curve   [date]   (value)

In [377]:
close_df.hvplot()

:NdOverlay   [Variable]
   :Curve   [date]   (value)

In [378]:
ETH_LTC_close_df.hvplot()

:NdOverlay   [Variable]
   :Curve   [date]   (value)

In [379]:
daily_change_df.hvplot()

:NdOverlay   [Variable]
   :Curve   [date]   (value)

# Created hypothetical tabs for dashboard

In [380]:
tab_1 = pn.Column(
    volume_change(),
    volatility(),
    close(),
    zoom_in_close(),
    pct_change_plot()
)
dashboard = pn.Tabs(
    ('tab_1', tab_1)
)

In [381]:
dashboard.servable()

Tabs
    [0] Column
        [0] HoloViews(NdOverlay)
        [1] HoloViews(NdOverlay)
        [2] HoloViews(NdOverlay)
        [3] HoloViews(NdOverlay)
        [4] HoloViews(NdOverlay)

# Pull twitter data, then clean and create merged dataset

In [382]:

twitter_path = Path('../data/raw_data/raw_tweets_01.csv')
twitter_df = pd.read_csv(twitter_path, infer_datetime_format=False, parse_dates=True)
datetime = twitter_df['time'].str.split(" ", n=1, expand = True)
twitter_df['date'] = datetime[0]
twitter_df.drop(columns='time')
twitter_df['time'] = datetime[1]
twitter_df.drop_duplicates(subset='text', inplace=True)
twitter_df.set_index('date', inplace=True)

df = pd.merge(twitter_df, volume_change_df, how='inner', left_index=True, right_index=True)
df2= pd.merge(df, volatility_df, how='inner', left_index=True, right_index=True)
df3 = pd.merge(df2, close_df, how='inner', left_index=True, right_index=True)
twitter_analysis_df = pd.merge(df3, daily_change_df, how='inner', left_index=True, right_index=True)
twitter_analysis_df.sort_index(inplace=True)
twitter_analysis_df['category'].fillna('null', inplace=True)
#tweet_analysis_df['2020-03-30':'2021-19-01']
#df1.merge(df2, on='ID', how='left')

# Here is some example data. We can add any tables and mess with the data from here. Anything we want to add on we should do through our crypto data frames and restart kernel

In [383]:
twitter_users = twitter_analysis_df['twitter_user'].unique()
crypto_category = twitter_analysis_df['category'].unique()
crypto_category
bitcoin_tweets_df = twitter_analysis_df[twitter_analysis_df['category'].str.contains('bit')]
ethereum_tweets_df = twitter_analysis_df[twitter_analysis_df['category'].str.contains('eth')]

In [384]:
elon = twitter_users[1]

In [385]:
elon_tweets_df = twitter_analysis_df[twitter_analysis_df['twitter_user'] == elon]

In [386]:
elon_tweets_df.head(2)

,twitter_user,category,time,sentiment,text,tweet_id,tweet_source,quote_count,reply_count,retweet_count,...,LTC_volume_change,BTC_volatility,ETH_volatility,LTC_volatility,BTC_close,ETH_close,LTC_close,BTC_daily_change,ETH_daily_change,LTC_daily_change
date,,,,,,,,,,,,,,,,,,,,,
2020-05-01,elonmusk,bitcoin,22:44:56+00:00,NaN,@Bitcoin How much for some anime Bitcoin? http...,1256353943765921792,Twitter for iPhone,367,713,2213,...,-0.005658,4.99,5.24,4.14,8829.42,212.09,47.11,-0.01738,-0.01008,-0.04365
2020-05-01,elonmusk,bitcoin,22:34:24+00:00,NaN,@Bitcoin 🤣🤣,1256351293007736832,Twitter for iPhone,25,163,176,...,-0.005658,4.99,5.24,4.14,8829.42,212.09,47.11,-0.01738,-0.01008,-0.04365


# Elon has limited tweets, so here is 

In [387]:
elon_tweets_df.hvplot.scatter(x='date', y='BTC_volume_change')

:Scatter   [date]   (BTC_volume_change)

In [388]:
mcafee = twitter_users[0]

In [389]:
mcafee_tweets_df = twitter_analysis_df[twitter_analysis_df['twitter_user'] == mcafee]

In [390]:

mcafee_tweets_df.head(2)

,twitter_user,category,time,sentiment,text,tweet_id,tweet_source,quote_count,reply_count,retweet_count,...,LTC_volume_change,BTC_volatility,ETH_volatility,LTC_volatility,BTC_close,ETH_close,LTC_close,BTC_daily_change,ETH_daily_change,LTC_daily_change
date,,,,,,,,,,,,,,,,,,,,,
2020-03-28,officialmcafee,generic,16:26:26+00:00,NaN,@crypto_lord1 No shit????? \nBummer,1243937504157458432,Twitter for Android,0,4,0,...,1.072342,5.37,6.57,7.44,6251.82,131.35,38.91,0.06374,0.05629,0.04850
2020-03-29,officialmcafee,generic,12:19:25+00:00,NaN,"RT @officialmcafee: $2 trillion printed to ""Ba...",1244237726402187264,Twitter for Android,0,0,0,...,-0.159512,6.52,6.40,5.56,5877.21,124.35,37.11,-0.08260,-0.06023,-0.04822


In [393]:
mcafee_tweets_df.hvplot(x='date', y='BTC_volume_change')

:Curve   [date]   (BTC_volume_change)

In [394]:
twitter_analysis_df.reset_index(inplace=True)
option_1 = twitter_analysis_df.set_index(['twitter_user', 'date'])
option_1.head()

index category            time  sentiment  \
twitter_user   date                                                    
officialmcafee 2020-03-28      0  generic  16:26:26+00:00        NaN   
               2020-03-29      1  generic  12:19:25+00:00        NaN   
               2020-03-29      2     null  06:37:46+00:00        NaN   
               2020-03-30      3  generic  08:03:20+00:00        NaN   
               2020-03-31      4  generic  10:36:30+00:00        NaN   

                                                                        text  \
twitter_user   date                                                            
officialmcafee 2020-03-28               @crypto_lord1 No shit?????  \nBummer   
               2020-03-29  RT @officialmcafee: $2 trillion printed to "Ba...   
               2020-03-29  $2 trillion printed to "Bail Out" America.\n\n...   
               2020-03-30                                  @nino_crypto Why?   
               2020-03-31  @nino_crypto Neither.\n\nWe're watching short ...   

                                      tweet_id         tweet_source  \
twitter_user   date                                                   
officialmcafee 2020-03-28  1243937504157458432  Twitter for Android   
               2020-03-29  1244237726402187264  Twitter for Android   
               2020-03-29  1244151749151244290  Twitter for Android   
               2020-03-30  1244535667671138304  Twitter for Android   
               2020-03-31  1244936602268434432  Twitter for Android   

                           quote_count  reply_count  retweet_count  ...  \
twitter_user   date                                                 ...   
officialmcafee 2020-03-28            0            4              0  ...   
               2020-03-29            0            0              0  ...   
               2020-03-29           31          223            370  ...   
               2020-03-30            0            1              0  ...   
               2020-03-31            0            2              0  ...   

                           LTC_volume_change  BTC_volatility  ETH_volatility  \
twitter_user   date                                                            
officialmcafee 2020-03-28           1.072342            5.37            6.57   
               2020-03-29          -0.159512            6.52            6.40   
               2020-03-29          -0.159512            6.52            6.40   
               2020-03-30           0.696666           11.74            8.44   
               2020-03-31          -0.343133            2.93            3.62   

                           LTC_volatility  BTC_close  ETH_close  LTC_close  \
twitter_user   date                                                          
officialmcafee 2020-03-28            7.44    6251.82     131.35      38.91   
               2020-03-29            5.56    5877.21     124.35      37.11   
               2020-03-29            5.56    5877.21     124.35      37.11   
               2020-03-30            6.76    6406.40     132.32      38.99   
               2020-03-31            2.61    6424.35     133.08      39.15   

                           BTC_daily_change  ETH_daily_change  \
twitter_user   date                                             
officialmcafee 2020-03-28           0.06374           0.05629   
               2020-03-29          -0.08260          -0.06023   
               2020-03-29          -0.08260          -0.06023   
               2020-03-30          -0.00279          -0.00571   
               2020-03-31          -0.03627          -0.02248   

                           LTC_daily_change  
twitter_user   date                          
officialmcafee 2020-03-28           0.04850  
               2020-03-29          -0.04822  
               2020-03-29          -0.04822  
               2020-03-30          -0.00409  
               2020-03-31          -0.00458  

[5 rows x 23 columns]

In [395]:
option_2 = twitter_analysis_df.set_index(['date', 'twitter_user'])
option_2.head()

index category            time  sentiment  \
date       twitter_user                                                
2020-03-28 officialmcafee      0  generic  16:26:26+00:00        NaN   
2020-03-29 officialmcafee      1  generic  12:19:25+00:00        NaN   
           officialmcafee      2     null  06:37:46+00:00        NaN   
2020-03-30 officialmcafee      3  generic  08:03:20+00:00        NaN   
2020-03-31 officialmcafee      4  generic  10:36:30+00:00        NaN   

                                                                        text  \
date       twitter_user                                                        
2020-03-28 officialmcafee               @crypto_lord1 No shit?????  \nBummer   
2020-03-29 officialmcafee  RT @officialmcafee: $2 trillion printed to "Ba...   
           officialmcafee  $2 trillion printed to "Bail Out" America.\n\n...   
2020-03-30 officialmcafee                                  @nino_crypto Why?   
2020-03-31 officialmcafee  @nino_crypto Neither.\n\nWe're watching short ...   

                                      tweet_id         tweet_source  \
date       twitter_user                                               
2020-03-28 officialmcafee  1243937504157458432  Twitter for Android   
2020-03-29 officialmcafee  1244237726402187264  Twitter for Android   
           officialmcafee  1244151749151244290  Twitter for Android   
2020-03-30 officialmcafee  1244535667671138304  Twitter for Android   
2020-03-31 officialmcafee  1244936602268434432  Twitter for Android   

                           quote_count  reply_count  retweet_count  ...  \
date       twitter_user                                             ...   
2020-03-28 officialmcafee            0            4              0  ...   
2020-03-29 officialmcafee            0            0              0  ...   
           officialmcafee           31          223            370  ...   
2020-03-30 officialmcafee            0            1              0  ...   
2020-03-31 officialmcafee            0            2              0  ...   

                           LTC_volume_change  BTC_volatility  ETH_volatility  \
date       twitter_user                                                        
2020-03-28 officialmcafee           1.072342            5.37            6.57   
2020-03-29 officialmcafee          -0.159512            6.52            6.40   
           officialmcafee          -0.159512            6.52            6.40   
2020-03-30 officialmcafee           0.696666           11.74            8.44   
2020-03-31 officialmcafee          -0.343133            2.93            3.62   

                           LTC_volatility  BTC_close  ETH_close  LTC_close  \
date       twitter_user                                                      
2020-03-28 officialmcafee            7.44    6251.82     131.35      38.91   
2020-03-29 officialmcafee            5.56    5877.21     124.35      37.11   
           officialmcafee            5.56    5877.21     124.35      37.11   
2020-03-30 officialmcafee            6.76    6406.40     132.32      38.99   
2020-03-31 officialmcafee            2.61    6424.35     133.08      39.15   

                           BTC_daily_change  ETH_daily_change  \
date       twitter_user                                         
2020-03-28 officialmcafee           0.06374           0.05629   
2020-03-29 officialmcafee          -0.08260          -0.06023   
           officialmcafee          -0.08260          -0.06023   
2020-03-30 officialmcafee          -0.00279          -0.00571   
2020-03-31 officialmcafee          -0.03627          -0.02248   

                           LTC_daily_change  
date       twitter_user                      
2020-03-28 officialmcafee           0.04850  
2020-03-29 officialmcafee          -0.04822  
           officialmcafee          -0.04822  
2020-03-30 officialmcafee          -0.00409  
2020-03-31 officialmcafee          -0.00458  

[5 rows x 23 columns]

In [396]:
bitcoin_tweets_df.hvplot.scatter(x='date', y='BTC_daily_change')

:Scatter   [date]   (BTC_daily_change)

In [397]:
ethereum_tweets_df.hvplot.scatter(x='date', y='ETH_daily_change')

:Scatter   [date]   (ETH_daily_change)